In [1]:
import tensorflow as tf
from tensorflow.contrib import seq2seq
import numpy as np

In [2]:
class Model:
    def __init__(
        self,
        vocabulary_size,
        maxlen = 50,
        output_size = 512,
        learning_rate = 1e-3,
        embedding_size = 256,
        batch_size = 16,
        max_grad_norm = 10,
        **kwargs
    ):
        word_embeddings = tf.Variable(
            tf.random_uniform(
                [vocabulary_size, embedding_size], -np.sqrt(3), np.sqrt(3)
            )
        )
        self.output_size = output_size
        self.maxlen = maxlen
        self.embeddings = word_embeddings
        self.output_layer = tf.layers.Dense(vocabulary_size)
        self.output_layer.build(output_size)

        self.BEFORE = tf.placeholder(tf.int32, [None, maxlen])
        self.INPUT = tf.placeholder(tf.int32, [None, maxlen])
        self.AFTER = tf.placeholder(tf.int32, [None, maxlen])
        self.batch_size = tf.shape(self.INPUT)[0]

        self.get_thought = self.thought(self.INPUT)
        self.attention = tf.matmul(
            self.get_thought, tf.transpose(self.embeddings), name = 'attention'
        )
        self.fw_logits = self.decoder(self.get_thought, self.AFTER)
        self.bw_logits = self.decoder(self.get_thought, self.BEFORE)
        self.loss = self.calculate_loss(
            self.fw_logits, self.AFTER
        ) + self.calculate_loss(self.bw_logits, self.BEFORE)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(
            tf.gradients(self.loss, tvars), max_grad_norm
        )
        self.optimizer = tf.train.AdamOptimizer(learning_rate).apply_gradients(
            zip(grads, tvars)
        )

    def get_embedding(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def thought(self, inputs):
        encoder_in = self.get_embedding(inputs)
        fw_cell = tf.nn.rnn_cell.GRUCell(self.output_size)
        bw_cell = tf.nn.rnn_cell.GRUCell(self.output_size)
        sequence_length = tf.reduce_sum(tf.sign(inputs), axis = 1)
        with tf.variable_scope(
            'thought_scope', reuse = False
        ):
            rnn_output = tf.nn.bidirectional_dynamic_rnn(
                fw_cell,
                bw_cell,
                encoder_in,
                sequence_length = sequence_length,
                dtype = tf.float32,
            )[1]
            return sum(rnn_output)

    def decoder(self, thought, labels):
        main = tf.strided_slice(labels, [0, 0], [self.batch_size, -1], [1, 1])
        shifted_labels = tf.concat([tf.fill([self.batch_size, 1], 2), main], 1)
        decoder_in = self.get_embedding(shifted_labels)
        cell = tf.nn.rnn_cell.GRUCell(self.output_size)
        max_seq_lengths = tf.fill([self.batch_size], self.maxlen)
        helper = seq2seq.TrainingHelper(
            decoder_in, max_seq_lengths, time_major = False
        )
        decoder = seq2seq.BasicDecoder(cell, helper, thought)
        decoder_out = seq2seq.dynamic_decode(decoder)[0].rnn_output
        return decoder_out

    def calculate_loss(self, outputs, labels):
        mask = tf.cast(tf.sign(labels), tf.float32)
        logits = self.output_layer(outputs)
        return seq2seq.sequence_loss(logits, labels, mask)

In [3]:
import json
with open('skip-news-dict.json') as fopen:
    dictionary = json.load(fopen)
len(dictionary)

54718

In [4]:
def rename(checkpoint_dir, replace_from, replace_to, add_prefix, dry_run=False):
    checkpoint = tf.train.get_checkpoint_state(checkpoint_dir)
    with tf.Session() as sess:
        for var_name, _ in tf.contrib.framework.list_variables(checkpoint_dir):
            var = tf.contrib.framework.load_variable(checkpoint_dir, var_name)
            new_name = var_name
            if None not in [replace_from, replace_to]:
                new_name = new_name.replace(replace_from, replace_to)
            if add_prefix:
                new_name = add_prefix + new_name

            if dry_run:
                print('%s would be renamed to %s.' % (var_name, new_name))
            else:
                print('Renaming %s to %s.' % (var_name, new_name))
                # Rename the variable
                var = tf.Variable(var, name=new_name)

        if not dry_run:
            # Save the variables
            saver = tf.train.Saver()
            sess.run(tf.global_variables_initializer())
            saver.save(sess, 'skip-rename/model.ckpt')

In [5]:
# rename('skip/model.ckpt','thought_scope_e1d42da4-5ae4-4898-b0f1-f52f687a4e28',
#       'thought_scope',None)

In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(len(dictionary), embedding_size = 128, output_size = 128, batch_size=16,maxlen=100)
sess.run(tf.global_variables_initializer())

In [7]:
[i.name for i in tf.global_variables()]

['Variable:0',
 'dense/kernel:0',
 'dense/bias:0',
 'thought_scope/bidirectional_rnn/fw/gru_cell/gates/kernel:0',
 'thought_scope/bidirectional_rnn/fw/gru_cell/gates/bias:0',
 'thought_scope/bidirectional_rnn/fw/gru_cell/candidate/kernel:0',
 'thought_scope/bidirectional_rnn/fw/gru_cell/candidate/bias:0',
 'thought_scope/bidirectional_rnn/bw/gru_cell/gates/kernel:0',
 'thought_scope/bidirectional_rnn/bw/gru_cell/gates/bias:0',
 'thought_scope/bidirectional_rnn/bw/gru_cell/candidate/kernel:0',
 'thought_scope/bidirectional_rnn/bw/gru_cell/candidate/bias:0',
 'decoder/gru_cell/gates/kernel:0',
 'decoder/gru_cell/gates/bias:0',
 'decoder/gru_cell/candidate/kernel:0',
 'decoder/gru_cell/candidate/bias:0',
 'decoder_1/gru_cell/gates/kernel:0',
 'decoder_1/gru_cell/gates/bias:0',
 'decoder_1/gru_cell/candidate/kernel:0',
 'decoder_1/gru_cell/candidate/bias:0',
 'beta1_power:0',
 'beta2_power:0',
 'Variable/Adam:0',
 'Variable/Adam_1:0',
 'dense/kernel/Adam:0',
 'dense/kernel/Adam_1:0',
 'den

In [8]:
saver=tf.train.Saver(tf.global_variables())
saver.restore(sess, 'skip/model.ckpt')

INFO:tensorflow:Restoring parameters from skip/model.ckpt


In [9]:
import random

def sequence(s, w2v_model, maxlen, vocabulary_size):
    words = s.split()
    np_array = np.zeros((maxlen),dtype=np.int32)
    current_no = 0
    for no, word in enumerate(words[:maxlen - 2]):
        id_to_append = 1
        if word in w2v_model:
            word_id = w2v_model[word]
            if word_id < vocabulary_size:
                id_to_append = word_id
        np_array[no] = id_to_append
        current_no = no
    np_array[current_no + 1] = 3
    return np_array

def generate_batch(sentences,batch_size,w2v_model,maxlen,vocabulary_size):
    window_size = batch_size + 2
    first_index = 1000
    batch_sentences = sentences[first_index:first_index+window_size]
    print(batch_sentences)
    batch_sequences = np.array([sequence(sentence,w2v_model,maxlen,vocabulary_size) for sentence in batch_sentences])
    window_shape = []
    for i in range(batch_size):
        window_shape.append(batch_sequences[i:i+3])
    window_shape = np.array(window_shape)
    return window_shape[:,0], window_shape[:,1], window_shape[:,2]

In [10]:
import json
with open('news-bm.json','r') as fopen:
    sentences = json.loads(fopen.read())

In [11]:
bw_input, current_input, fw_input = generate_batch(sentences,1,dictionary,100,len(dictionary))

['pahang diwakili pemangku raja pahang tengku abdullah sultan ahmad shah manakala kelantan diwakili pemangku raja kelantan dr', 'tengku muhammad faiz petra', 'pada hari kedua mesyuarat yang bermula kira pukul pagi itu raja-raja melayu diiringi menteri besar masing-masing manakala yang dipertua negeri pulau pinang sabah dan melaka diiringi ketua menteri masing-masing']


In [12]:
encoded = sess.run(model.get_thought,feed_dict={model.INPUT:fw_input})

In [13]:
encoded

array([[ 1.0416520e-01, -5.5048943e-01, -8.6489022e-01, -4.7374249e-02,
         1.1276997e+00,  1.8109307e+00,  6.9022512e-01,  1.3390839e-02,
         2.2568166e-01, -1.2908951e+00,  1.8937750e+00, -6.6073686e-01,
         8.8402975e-01, -1.9575896e+00, -1.3369490e+00,  8.7181759e-01,
         6.0808134e-01, -1.3946321e+00,  1.2038462e-01,  1.2153907e+00,
         5.5231041e-01, -1.6721604e+00, -1.9526482e-04, -6.4797735e-01,
         1.9013047e-02,  1.6876624e+00, -1.7706637e+00,  3.0935839e-01,
         2.3643266e-01, -7.0005804e-01, -7.6473856e-01, -6.4990938e-01,
         8.5101128e-02,  1.9995425e+00, -1.3742411e+00,  1.4046657e+00,
         1.2373401e+00,  1.3037590e+00,  5.5078387e-01, -1.6784103e+00,
        -1.5637214e+00,  1.4834172e-01, -1.0372441e+00, -2.6549307e-01,
        -1.8813536e+00,  1.2753011e-01,  1.6532394e+00, -5.8884758e-01,
        -2.4680305e-01, -1.9865925e+00,  7.4487889e-01, -2.9214048e-01,
         7.9541242e-01, -7.1536422e-01,  9.7346407e-01, -2.97804

In [14]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if (
            'Variable' in n.op
            or n.name.find('Placeholder') >= 0
            or 'add_1' in n.name
            or 'attention' in n.name
        )
        and 'Adam' not in n.name
    ]
)

In [15]:
strings.split(',')

['Variable',
 'dense/kernel',
 'dense/bias',
 'Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'thought_scope/bidirectional_rnn/fw/gru_cell/gates/kernel',
 'thought_scope/bidirectional_rnn/fw/gru_cell/gates/bias',
 'thought_scope/bidirectional_rnn/fw/gru_cell/candidate/kernel',
 'thought_scope/bidirectional_rnn/fw/gru_cell/candidate/bias',
 'thought_scope/bidirectional_rnn/fw/fw/while/add_1/y',
 'thought_scope/bidirectional_rnn/fw/fw/while/add_1',
 'thought_scope/bidirectional_rnn/bw/gru_cell/gates/kernel',
 'thought_scope/bidirectional_rnn/bw/gru_cell/gates/bias',
 'thought_scope/bidirectional_rnn/bw/gru_cell/candidate/kernel',
 'thought_scope/bidirectional_rnn/bw/gru_cell/candidate/bias',
 'thought_scope/bidirectional_rnn/bw/bw/while/add_1/y',
 'thought_scope/bidirectional_rnn/bw/bw/while/add_1',
 'thought_scope/add_1',
 'attention',
 'decoder/gru_cell/gates/kernel',
 'decoder/gru_cell/gates/bias',
 'decoder/gru_cell/candidate/kernel',
 'decoder/gru_cell/candidate/bias',
 'decoder

In [16]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [17]:
freeze_graph('skip', strings)

INFO:tensorflow:Restoring parameters from skip/model.ckpt
INFO:tensorflow:Froze 21 variables.
INFO:tensorflow:Converted 21 variables to const ops.
1224 ops in the final graph.


In [18]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [19]:
g=load_graph('skip/frozen_model.pb')

In [20]:
x = g.get_tensor_by_name('import/Placeholder_1:0')
logits = g.get_tensor_by_name('import/thought_scope/add_1:0')
attention = g.get_tensor_by_name('import/attention:0')
test_sess = tf.InteractiveSession(graph=g)
out, att = test_sess.run([logits,attention], feed_dict={x:fw_input})

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [21]:
att.shape

(1, 54718)

In [23]:
rev_dict = {v: k for k, v in dictionary.items()}

In [24]:
for i in att[0].argsort()[-10:][::-1]:
    print(i)
    print(rev_dict[i])

49104
menjebaknya
54
seperti
5951
gunanya
41221
hawar
6333
ganjaran
27612
dayangku
33504
pijak
44119
parol
43996
poupart
22753
scb
